In [2]:
#!pip install opencv-python

     |████████████████████████████████| 52.2 MB 43.9 MB/s eta 0:00:01


In [3]:
import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.models import load_model

In [4]:
XMIN, XMAX = 100, 100 + 280
YMIN, YMAX = 100, 100 + 280

In [5]:
def preprocess_image(pic):
    im = Image.fromarray(pic)
    im = im.convert('L')
    im = im.resize((28, 28))
    #im.save(f'image.png')
    X = np.array(im)
    X = 255 - X
    X = (X-X.min()).astype(np.float32)
    X = X / X.max()
    X[X<0.25] = 0.0
    X = X.reshape(1, 28, 28, 1)
    return X

In [6]:
m = load_model("model.h5")

OSError: SavedModel file does not exist at: model.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
# start the camera
cap = cv2.VideoCapture(0)

In [ ]:
# count frames
i = 0
ypred = np.zeros(10)

In [ ]:
while(True):
    # Capture a single frame (code from OpenCV doc)
    ret, frame = cap.read()
    
    i += 1
    if i % 10 == 0:
        # predict only every 10th frame
        pic = frame[YMIN:YMAX, XMIN:XMAX]
        X = preprocess_image(pic)
        ypred = m.predict(X)
        
    # draw frame
    frame[YMIN:YMIN+2, XMIN:XMAX, :] = 255
    frame[YMAX:YMAX+2, XMIN:XMAX, :] = 255
    frame[YMIN:YMAX, XMIN:XMIN+2, :] = 255
    frame[YMIN:YMAX, XMAX:XMAX+2, :] = 255
    
    for j in range(10):
        xpos = XMAX + 50
        ypos = j*35 + YMIN
        ypred = ypred.flatten()
        width = int(ypred[j] * 80)
        cv2.putText(frame, str(j),
                    (xpos, ypos),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 255, 255, 255), 3) # size, color, stroke
        frame[ypos-5:ypos+5, xpos+30:xpos+30+width, :] = 255
        
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): # quit on 'q'
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()